In [1]:
import base64
import os

from pypdfium2 import PdfDocument
from langchain_core.messages import HumanMessage
from unstructured.partition.pdf import partition_pdf
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_core.messages import AIMessage
from langchain_core.runnables import RunnableLambda

In [2]:
pdf_name = 'ref_data/TWINGO/MR413X4416A000.pdf'

In [3]:
# pdf = PdfDocument(pdf_name)
# for i in range(len(pdf)):
#     page = pdf[i]
#     image = page.render(scale=4).to_pil()
#     image.save(f"ref_data/parsed_images/page_{i}.jpg")

In [3]:
images_path = "ref_data/parsed_images/"
pdf_elements = partition_pdf(
    pdf_name,
    chunking_strategy="by_title",
    extract_images_in_pdf=True,
    max_characters=3000,
    new_after_n_chars=2800,
    combine_text_under_n_chars=2000,
    image_output_dir_path=images_path,
    )

In [4]:
tables = []
texts = []
for element in pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        tables.append(str(element))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        texts.append(str(element))

In [10]:
prompt_text = """You are an assistant tasked with summarizing tables and text for retrieval. \
These summaries will be embedded and used to retrieve the raw text or table elements. \
Give a concise summary of the table or text that is well-optimized for retrieval. Table \
or text: {element} """
prompt = PromptTemplate.from_template(prompt_text)
empty_response = RunnableLambda(
    lambda x: AIMessage(content="Error processing document")
)
model = ChatOpenAI(model="gpt-4o-mini")
summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

text_summaries = []
table_summaries = []
text_summaries = summarize_chain.batch(texts, {"max_concurrency": 1})
table_summaries = summarize_chain.batch(tables, {"max_concurrency": 1})

In [13]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


def image_summarize(img_base64, prompt):
    model = ChatOpenAI(model="gpt-4o-mini")
    msg = model(
        [
            HumanMessage(
                content=[
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{img_base64}"},
                    },
                ]
            )
        ]
    )
    return msg.content

img_base64_list = []
images_summaries = []

# Prompt
prompt = """You are an assistant tasked with summarizing images for retrieval. \
These summaries will be embedded and used to retrieve the raw image. \
Give a concise summary of the image that is well optimized for retrieval."""

# Apply to images
for img_file in sorted(os.listdir(images_path)):
    if img_file.endswith(".jpg"):
        img_path = os.path.join(images_path, img_file)
        base64_image = encode_image(img_path)
        img_base64_list.append(base64_image)
        images_summaries.append(image_summarize(base64_image, prompt))

In [15]:
texts

['TUVUIN GO\n\n1 Двигатель и его системы\n\nЗАПУСК ДВИГАТЕЛЯ И ЗАРЯДКА АККУМУЛЯТОРНОЙ БАТАРЕИ Twingo II Диагностика – Перечень и расположение элементов 16A - 2 Диагностика – Назначение элементов системы 16A - 3 16A - 4 Диагностика – Функциональная схема 16A - 5 Диагностика – Выполняемые функции 16A - 6 Диагностика – Жалобы владельцев Диагностика – Алгоритмы поиска неисправностей Диагностика – Проверки 16A - 7 16A - 16\n\n16A\n\nV2 Edition Russe "Методы ремонта, рекомендуемые изготовителем в настоящем документе, соответствуют техническим условиям, действительным на момент составления руководства. В случае внесения конструктивных изменений в изготовление деталей, узлов, агрегатов автомобиля данной модели, методы ремонта могут быть также соответственно изменены". Все авторские права принадлежат Renault s.a.s. Воспроизведение или перевод, в том числе частичные, настоящего документа, равно как и использование системы нумерации запасных частей, запрещены без предварительного письменного разр

In [14]:
text_summaries

['**Summary of TUVUIN GO Table:**\n\nThe table provides diagnostic information for the engine and systems of the Twingo II, focusing on engine start and battery charging. It includes elements like the starter and generator, along with their locations and functions. The document is a repair guide by Renault that outlines diagnostic procedures, troubleshooting algorithms, and checks, emphasizing adherence to technical conditions. It also notes copyright restrictions on reproduction and translation of the material. \n\n**Key Topics:**\n- Engine start and battery charging diagnostics\n- Element location and function\n- Troubleshooting and checks\n- Copyright information by Renault s.a.s.',
 "**Summary of Engine Start and Battery Charging System for Twingo II**\n\nThe system comprises four main components: \n\n1. **Battery**: Provides a powerful energy impulse for engine start and powers essential systems when the engine is off, such as alarms and audio systems.\n2. **Generator**: Operates 

In [7]:
tables

['АПН 1 Стартер не функционирует УКАЗАНИЯ Выполните проверку состояния аккумуляторной батареи (см. главу 80A, Аккумуляторная батарея). Выполните проверку состояния силового предохранителя (см. ПРОВЕРКА 2). Проверьте исправность предохранителя F9 компонента 1016. Проверьте, не заклинило ли двигатель. Проверьте надежность затяжки следующих наконечников: ⇒ аккумуляторную батарею, ⇒ стартера ⇒ "массы" двигателя ⇒ "массы" кузова Наконечники проводов надежно затянуты? НЕТ Затяните клеммы (см. Руководство по ремонту 411, глава 80A, Аккумуляторная батарея, Аккумуляторная батарея: Снятие и установка). ДА Проверьте исправность следующих проводов: ⇒ провод, соединяющий вывод "+" аккумуляторной батареи, компонент 107, с контактом B "+" стартера, компонент 163, ⇒ провод, соединяющий вывод "-" аккумуляторной батареи с "массой" кузова автомобиля, ⇒ провод, соединяющий двигатель с "массой" кузова автомобиля. Провода в порядке? ДА НЕТ Устраните неисправность поврежденных проводов и их наконечников. Есл

In [16]:
table_summaries

["**Summary: Troubleshooting Starter Issues (АПН 1)**\n\n1. **Check Battery**: Verify the battery's condition (refer to Chapter 80A).\n2. **Inspect Fuse**: Check the power fuse and the F9 fuse for component 1016.\n3. **Starter and Engine**: Ensure the starter is not jammed.\n4. **Tighten Connections**: Confirm tightness of connections for:\n   - Battery\n   - Starter\n   - Engine ground\n   - Chassis ground\n5. **Wire Inspection**: If connections are tight, check wires:\n   - From battery positive to starter\n   - From battery negative to chassis\n   - From engine to chassis\n6. **Repair or Replace Wires**: Fix any damaged wires; replace if irreparable.\n7. **Attempt Engine Start**: If the starter does not turn, check for a clicking sound when engaged.\n\n**Note**: Consult relevant repair guides for detailed instructions.",
 '**Summary**: The text outlines a diagnostic procedure for troubleshooting a starter relay control circuit with fault code DEF 062. It includes steps to check for 

In [17]:
images_summaries

['Document titled "Twingo II" outlining engine diagnostics and battery charging procedures in Russian. Sections include engine start, battery charging, component lists, functional diagrams, owner complaints, troubleshooting algorithms, and checks. Edition dated 2008.',
 'Technical illustration of a Twingo II engine, labeled with components related to the starter (1), generator (2), and a wiring diagram (3). Includes diagnostic information in Russian regarding engine start and battery charging systems.',
 'A technical diagnostic flowchart for vehicle engine start and battery charging issues, outlining steps to check voltage, identify potential faults in wiring components 232 and 163, and instructions for repair or replacement of parts. Includes references to repair manuals and safety measures.',
 'Flowchart detailing troubleshooting steps for a vehicle starter issue. It includes instructions for disconnecting the battery, checking the starter and flywheel, and replacing components if ne